# 05 Advanced Model Poisoning Attacks and Adaptive Aggregation Strategies

This notebook extends the Federated Learning-based Intrusion Detection System (FL-IDS) for Industrial IoT (IIoT) by implementing advanced model update poisoning attacks and adaptive aggregation strategies.

---

### Objectives:
- **Simulate Advanced Model Update Poisoning Attacks:**
  - Sign Flipping Attack
  - Scaling Attack
  - Sybil Attack (Coordinated Attackers)
  - Adaptive Gradient-based Attack

- **Implement Adaptive Aggregation Strategies:**
  - Drift-Aware Weighting of Client Updates
  - Performance Feedback-based Aggregation

- **Simulate Realistic FL Scenarios:**
  - Client Dropout Simulation
  - Data and Computational Heterogeneity

- **Evaluate System Resilience under Adversarial Conditions**
  - Impact on Global Autoencoder Performance
  - Comparative Analysis with Robust Aggregation Methods



In [1]:
import os
import numpy as np
import pandas as pd
import random
import tensorflow as tf
from tensorflow.keras.models import load_model, clone_model
from tensorflow.keras.losses import MeanSquaredError
import joblib

# For reproducibility
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)

# === Defining Paths ===
base_path = "D:/August-Thesis/FL-IDS-Surveillance"
results_path = os.path.join(base_path, "notebooks/results")

model_path = os.path.join(results_path, "models/unsupervised/federated/final_federated_autoencoder_20rounds.h5")
scaler_path = os.path.join(results_path, "scalers/minmax_scaler_client_3.pkl")  

data_path = os.path.join(base_path, "data/processed/federated/unsupervised")
test_path = os.path.join(base_path, "data/processed/surv_unsupervised/test_mixed.csv")

client_ids = [f"client_{i}" for i in range(1, 6)]

# === Loading the model (uncompiled) ===
global_model = load_model(model_path, compile=False)
global_model.compile(optimizer="adam", loss=MeanSquaredError())

# === Loading necessary Scaler ===
minmax_scaler = joblib.load(scaler_path)

feature_cols = list(minmax_scaler.feature_names_in_)


# === Load Client Data ===
client_dfs = {
    cid: pd.read_csv(os.path.join(data_path, cid, "train.csv")) 
    for cid in client_ids
}

# === Load Testing Set ===
test_df = pd.read_csv(test_path, low_memory = False)
X_test = test_df[feature_cols].astype(float)
y_true = test_df["Attack_label"].values
X_test_scaled = minmax_scaler.transform(test_df[feature_cols])

print("The setup is  complete:")
print(f"- Loaded model: {os.path.basename(model_path)}")
print(f"- Clients: {client_ids}")
print(f"- Test samples: {X_test.shape[0]}")


The setup is  complete:
- Loaded model: final_federated_autoencoder_20rounds.h5
- Clients: ['client_1', 'client_2', 'client_3', 'client_4', 'client_5']
- Test samples: 2218834


In [2]:
from sklearn.metrics import f1_score, precision_score, recall_score

# Use the best threshold from Notebook 03
threshold = 0.000639

# Baseline evaluation to establish 
reconstructed = global_model.predict(X_test_scaled, verbose=0)
reconstruction_errors = np.mean(np.square(X_test_scaled - reconstructed), axis=1)
y_pred = (reconstruction_errors > threshold).astype(int)

# the Metrics
f1_baseline = f1_score(y_true, y_pred)
precision_baseline = precision_score(y_true, y_pred)
recall_baseline = recall_score(y_true, y_pred)

print(f"Baseline Performance Before Attack:")
print(f"- F1 Score:      {f1_baseline:.4f}")
print(f"- Precision:     {precision_baseline:.4f}")
print(f"- Recall:        {recall_baseline:.4f}")


Baseline Performance Before Attack:
- F1 Score:      0.8373
- Precision:     0.9877
- Recall:        0.7266
